## 1. 掛載 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定工作目錄
import os
os.chdir('/content/drive/MyDrive/mosquito-training')

print("✓ Google Drive 已掛載")
print(f"✓ 工作目錄: {os.getcwd()}")

## 2. 安裝訓練環境

In [ ]:
!pip install ultralytics -q

# 檢查 GPU
import torch
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ 警告：未偵測到 GPU，請檢查執行階段設定")

## 3. 準備訓練數據集

In [ ]:
import os
import shutil
from pathlib import Path
import random

# 建立目錄結構
os.makedirs('training_dataset/images/train', exist_ok=True)
os.makedirs('training_dataset/images/val', exist_ok=True)
os.makedirs('training_dataset/labels/train', exist_ok=True)
os.makedirs('training_dataset/labels/val', exist_ok=True)

# 處理蚊子樣本
mosquito_dir = 'sample_collection/confirmed/mosquito'
mosquito_images = list(Path(mosquito_dir).glob('*.jpg'))

if len(mosquito_images) == 0:
    print("❌ 錯誤：找不到蚊子樣本圖片")
    print(f"請確認 {mosquito_dir} 目錄存在且包含圖片")
else:
    # 80/20 分割
    random.shuffle(mosquito_images)
    split_idx = int(len(mosquito_images) * 0.8)
    train_images = mosquito_images[:split_idx]
    val_images = mosquito_images[split_idx:]

    # 複製圖片並生成標籤
    for img_list, split in [(train_images, 'train'), (val_images, 'val')]:
        for img_path in img_list:
            # 複製圖片
            shutil.copy(img_path, f'training_dataset/images/{split}/')

            # 生成 YOLO 標籤（整張圖為蚊子）
            label_path = f'training_dataset/labels/{split}/{img_path.stem}.txt'
            with open(label_path, 'w') as f:
                f.write('0 0.5 0.5 1.0 1.0\n')  # class_id x_center y_center width height

    print(f"✓ 訓練集: {len(train_images)} 張")
    print(f"✓ 驗證集: {len(val_images)} 張")

    # 建立 dataset.yaml
    yaml_content = """path: /content/drive/MyDrive/mosquito-training/training_dataset
train: images/train
val: images/val
names:
  0: mosquito
nc: 1
"""

    with open('training_dataset/dataset.yaml', 'w') as f:
        f.write(yaml_content)

    print("✓ 數據集準備完成")

## 4. 開始訓練模型

In [ ]:
from ultralytics import YOLO

# 載入預訓練模型（從 Google Drive）
model = YOLO('models/mosquito_yolov8.pt')

# 訓練
results = model.train(
    data='training_dataset/dataset.yaml',
    epochs=50,
    imgsz=320,
    batch=16,
    lr0=0.001,
    name='mosquito_finetune',
    patience=10,
    device=0  # 使用 GPU
)

print("\n" + "="*50)
print("✓ 訓練完成!")
print(f"✓ 最佳模型: runs/detect/mosquito_finetune/weights/best.pt")
print("="*50)

## 5. 評估訓練結果

In [ ]:
# 驗證模型
metrics = model.val()

print("\n訓練指標：")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"精確率: {metrics.box.mp:.4f}")
print(f"召回率: {metrics.box.mr:.4f}")

## 6. 複製模型到 Google Drive

In [ ]:
import shutil
from datetime import datetime

# 複製最佳模型
shutil.copy(
    'runs/detect/mosquito_finetune/weights/best.pt',
    'models/mosquito_yolov8_new.pt'
)

# 同時備份一份帶日期的版本
date_str = datetime.now().strftime('%Y%m%d')
shutil.copy(
    'runs/detect/mosquito_finetune/weights/best.pt',
    f'models/mosquito_yolov8_{date_str}.pt'
)

print("✓ 新模型已保存到 Google Drive:")
print("  - models/mosquito_yolov8_new.pt")
print(f"  - models/mosquito_yolov8_{date_str}.pt")
print("\n請在本地同步 Google Drive 後，執行部署步驟")

## 7. (可選) 查看訓練曲線

In [ ]:
from IPython.display import Image, display

# 顯示訓練結果圖表
print("訓練曲線：")
display(Image('runs/detect/mosquito_finetune/results.png'))

print("\n混淆矩陣：")
display(Image('runs/detect/mosquito_finetune/confusion_matrix.png'))